In [ ]:
import sys
import os

# we want to import some llama source later
os.getcwd()
project_path = os.path.abspath("LLM")

if project_path not in sys.path:
    sys.path.append(project_path)

In [6]:
os.chdir('/home/matt/.llama/checkpoints/Llama3.2-1B')
os.getcwd()

'/home/matt/.llama/checkpoints/Llama3.2-1B'

In [7]:
os.chdir('/home/matt/.llama/checkpoints')

In [8]:
from transformers import AutoTokenizer, LlamaConfig, LlamaForCausalLM, AutoModelForCausalLM
import torch

break # It would be great to run this, but it OOMs with my meagre 8Gb of RAM

model_path = 'Llama3.2-1B'
#https://stackoverflow.com/a/78911943

# Load the tokenizer directly from the model path
tokenizer = AutoTokenizer.from_pretrained(model_path)
print("tokenizer loaded")

# Load model configuration from params.json
config = LlamaConfig.from_json_file(f'{model_path}/params.json')
print("config loaded")

# load the model with the specific configs. 
model = LlamaForCausalLM(config=config)
print("model loaded")

# Load the weights of the model
state_dict = torch.load(f'{model_path}/consolidated.00.pth', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
print("weights loaded")

model.eval()
print("eval called")

SyntaxError: 'break' outside loop (1351421061.py, line 4)

## Converting from the default file download format

When downloaded from llama.com, the files look like this

```
checklist.chk  config.json  consolidated.00.pth  params.json  tokenizer.model
```

We want them in the HuggingFace format, to do that I ran this script from the `transformers` package (included here for convenience)

```bash
python3 convert_llama_to_hf.py --input_dir /home/matt/.llama/checkpoints/Llama3.2-1B --model_size 1B --output_dir /home/matt/.llama/checkpoints/Llama3.2-1B-hf --llama_version 3.2
```

That then populates the output directory with the desired files, which look like:

```
config.json  generation_config.json  model.safetensors  special_tokens_map.json  tokenizer.json  tokenizer_config.json
```

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "Llama3.2-1B-hf"

tokenizer = AutoTokenizer.from_pretrained(model_path)
print("tokenizer loaded")

# load model with reduced precision
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",  # automatically uses float16/bfloat16 if available
    low_cpu_mem_usage=True,  # prevents high RAM usage
    device_map="auto"  # automatically assigns layers to GPU/CPU based on available memory
)

print("model loaded")


tokenizer loaded


Some parameters are on the meta device because they were offloaded to the disk and cpu.


model loaded


In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

input_text = "hello how are you?"

inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
inputs["attention_mask"] = (inputs["input_ids"] != tokenizer.pad_token_id).long()

with torch.no_grad():  # reduces memory usage
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=150,  
        temperature=1.0,
        pad_token_id=tokenizer.pad_token_id 
    )

output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output)

hello how are you? nice to meet you!
i'm a graphic designer with a passion for fine arts.
i'm interested in art, music, and people. i love animals, cooking, hiking, and traveling.
i've worked on both desktop and web projects. i'm currently interested in web design, but also have a passion for graphic design.
i'm interested in art, music, and people. i love animals, cooking, hiking, and traveling.
Hello! Nice to meet you.
I'm interested in art, music, and people. i love animals, cooking, hiking, and traveling.
i'm a passionate graphic designer and a self-taught web developer. i have worked in both desktop and web projects and have a


## Loading into `Transfomer` instance from source code

We are interested in the activations through the layers of this model, so it would be good to load create an instance of the `Transformer` object defined in the `model.py` file

In [10]:
import llama.model

ModuleNotFoundError: No module named 'fairscale'